In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [1]:
from fastai.text import *

In [2]:
PATH = Path('../../data/menu')
TMP_PATH = PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)
rawData = 'Dish.csv'
!ls '../../data/menu/tmp'

dishCodes.npy  wordDict.pkl


In [3]:
dfRaw = pd.read_csv((PATH/rawData).open())

In [35]:
def getTokenIdx(rows):
    freqs = Counter(wd for name in rows for wd in name.split(' ') )
    itos = [key for key, count in freqs.most_common()]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda:3, {v:k for k,v in enumerate(itos)})
    strCodes = [([stoi[i] for i in name.split(' ')]+[2]) for name in rows ]
    strCodes = np.array(strCodes)
    np.save(TMP_PATH/'dishCodes.npy', strCodes)
    pickle.dump(itos, open(TMP_PATH/'wordList.pkl','wb'))
    pickle.dump(dict(stoi), open(TMP_PATH/'wordDict.pkl','wb'))
    # freqFreq = Counter(count for key, count in freqs.most_common())
    # freqFreq.most_common()
    return strCodes, stoi, itos
strCodes, stoi, itos = getTokenIdx(dfRaw.name)

In [36]:
def loadTokenIdx():
    strCodes = np.load(TMP_PATH/'dishCodes.npy')
    itos = pickle.load(open(TMP_PATH/'wordList.pkl','rb'))
    stoi = collections.defaultdict(lambda: 3, pickle.load(open(TMP_PATH/'wordDict.pkl','rb')))
    return strCodes, stoi, itos
strCodes, stoi, itos = loadTokenIdx()